In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## BigQuery Magic Commands
Within Colab, there are a set of magic commands that allow us to quickly query BigQuery data and pass it into dataframes.

Below, I'll illustrate the syntax:

%%bigquery --project "project_name" "dataframe name"

%% -> This instantiates the magic command
-- -> This signifies that you're going to access a specific project
Pass in the project name and dataframe name where you'd like to output your data.

In [0]:
#example
%%bigquery --project xaxis-1 imps_df
SELECT 
      customer_link, count(customer_link) AS impressions,
      case when placement_id in (262151588,262474159,262474165,262566621,262566624,263896284) then 'CRM_Infant'
           when placement_id in (262152209,262152428,262474009,262474156,262474696,263896281) then 'CRM_Newborn'
          when placement_id in (262151594,262152206,262152425,262474168,262566627,263831179) then 'CRM_Non_Redeemer'
          when placement_id in (262193828,262637026,262637410,262716144,262717272) then 'Strong_moms_LAL'
          when placement_id in (262572137,262890418,262891312,262963467,262963689) then 'strong_moms_siteRTG'
          when placement_id in (262194401) then 'strong_moms_plista'
          when placement_id in (262474186,262472569,262469473,262561437,262474183,262474714,262152434,262152212,262151603,262152215) then 'meredith'
          else 'no_package' end as Tactic
          from `xaxis-1.Abbott.Similac_Impressions`
          GROUP BY customer_link, placement_id;

In [3]:
imps_df.head(10)

,customer_link,impressions,Tactic
0,XY6155hozXoWEkYlOEqa1MW18a20On8QUlQYj2PTOIvOscF3g,11,strong_moms_plista
1,XY61551E4LT5z_Uv3Gj_QfnNxxCzmMYDMBKZC8tMvNQS3y0SY,11,strong_moms_plista
2,XY6155tTsrARq-WDPK29pgr-3iZUx3UvK8XIwqSLev-_LRpfE,12,strong_moms_plista
3,XY6155w8QdT_8Ok2EgBZpF5jOawQBzduYBtUvHFK-ko3o6AoQ,11,strong_moms_plista
4,XY6155MB4c7QfpaxtcqYFLKvr1ZUHxiOxA5mEzKqdiB2XnF2s,12,strong_moms_plista
5,XY6155da62f0Oj5w3hJBGg6oYQmfLXnDRysqzzwE21drLlIBY,12,strong_moms_plista
6,Xi6155WZX0p2_hrPmUryka8xOuQTyeY1gTzJ3rAEo_pOBb...,13,strong_moms_plista
7,XY6155559e_tJK7jMq0odeZMtUWNzRXSPokqlQO58kvqMo1oQ,15,Strong_moms_LAL
8,Xi6155l3p7d-BuwvUq6g30tA7h-hXPtHToLG0HceGYpDdE...,15,Strong_moms_LAL
9,XY6155rDnQ8iSnrZj9IMCXfL6Sfab49Pwjwfu-hPp6UWJMTqw,17,Strong_moms_LAL


In [0]:
#example
%%bigquery --project xaxis-1 convs_df
SELECT customer_link, 
       count(customer_link) as conversions
FROM `xaxis-1.Abbott.Similac_CRM`
WHERE redeemer = true
GROUP BY 1;

In [5]:
convs_df.head(10)

,customer_link,conversions
0,XY6155TOb3PV_edw81wUj-O8UbU4zdGzzWPFJcA72da78g_WU,1
1,Xi6155KZTRl6imorv_TvYBhDfSvFPsjUNDS46cHlLBZwb8...,1
2,Xi615577l_pcxMavfWlDgIM0qvTw3YcJUeoXDckksZAazA...,1
3,XY6155kRXDxqb23G3qDXY3v05JnVzjZNCfLcrkkyJEcjkIBwk,1
4,XY6155Ooov_hAX0tp-50qiT1hxF58Hi7Ua5nGMoklC7UywkfA,1
5,XY6155e1Lugcs43DRhXpG5Fn84eSJJMOBXLvHOe0XbgumjSGk,1
6,XY6155eYi6EY9d2t8wDjDjVsTqXQXJM1CA0WsvF8erVs4se_8,1
7,XY61554g4gkbxfV5VRIXVHYd1kW-wJWy50qLX8cyBkEhGcmWs,1
8,XY6155ikdg5suncvWw6hAI5jMKEuO4lDF67tcT2psvB3HE9ys,1
9,XY6155RmWjiLkFDnvOVkmYKXVxO-OmPiVexeUCxd6cpMA8wJo,1


In [0]:
import pandas as pd
combined_df = pd.merge(imps_df, convs_df, how='left', on='customer_link')

In [7]:
combined_df.head()

,customer_link,impressions,Tactic,conversions
0,XY6155hozXoWEkYlOEqa1MW18a20On8QUlQYj2PTOIvOscF3g,11,strong_moms_plista,NaN
1,XY61551E4LT5z_Uv3Gj_QfnNxxCzmMYDMBKZC8tMvNQS3y0SY,11,strong_moms_plista,NaN
2,XY6155tTsrARq-WDPK29pgr-3iZUx3UvK8XIwqSLev-_LRpfE,12,strong_moms_plista,NaN
3,XY6155w8QdT_8Ok2EgBZpF5jOawQBzduYBtUvHFK-ko3o6AoQ,11,strong_moms_plista,NaN
4,XY6155MB4c7QfpaxtcqYFLKvr1ZUHxiOxA5mEzKqdiB2XnF2s,12,strong_moms_plista,NaN


In [8]:
pivot = pd.pivot_table(combined_df, index='Tactic', values=["impressions", "conversions"], aggfunc='sum').reset_index()
pivot

,Tactic,conversions,impressions
0,CRM_Infant,5370.0,903235
1,CRM_Newborn,13151.0,1738690
2,CRM_Non_Redeemer,4573.0,585349
3,Strong_moms_LAL,2751.0,1372811
4,meredith,2505.0,103161
5,strong_moms_plista,1057.0,1592322
6,strong_moms_siteRTG,3381.0,1313844


In [9]:
pivot['conversion_rate'] = pivot['conversions'] / pivot['impressions']
pivot

,Tactic,conversions,impressions,conversion_rate
0,CRM_Infant,5370.0,903235,0.005945
1,CRM_Newborn,13151.0,1738690,0.007564
2,CRM_Non_Redeemer,4573.0,585349,0.007812
3,Strong_moms_LAL,2751.0,1372811,0.002004
4,meredith,2505.0,103161,0.024282
5,strong_moms_plista,1057.0,1592322,0.000664
6,strong_moms_siteRTG,3381.0,1313844,0.002573


In [10]:
pivot['cr_100'] = pivot['conversion_rate'] * 100
pivot

,Tactic,conversions,impressions,conversion_rate,cr_100
0,CRM_Infant,5370.0,903235,0.005945,0.594530
1,CRM_Newborn,13151.0,1738690,0.007564,0.756374
2,CRM_Non_Redeemer,4573.0,585349,0.007812,0.781243
3,Strong_moms_LAL,2751.0,1372811,0.002004,0.200392
4,meredith,2505.0,103161,0.024282,2.428243
5,strong_moms_plista,1057.0,1592322,0.000664,0.066381
6,strong_moms_siteRTG,3381.0,1313844,0.002573,0.257336
